# Parquet Ingestion

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LineageLogic/LakeLogic/blob/main/examples/03_data_sources/files/parquet/parquet_demo.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/LineageLogic/LakeLogic/blob/main/examples/03_data_sources/files/parquet/parquet_demo.ipynb)

## Business Scenario

Parquet files arrive from upstream systems. You need consistent schema enforcement before analytics.

## Value Proposition

- Validate Parquet schema
- Enforce quality rules
- Prepare clean data

---

## Goals

1. Load a Parquet file
2. Validate against contract
3. Inspect output


## Step 2: Review the Contract

Open `parquet_contract.yaml` to see the schema, source config, and quality rules.


## Step 3: Run the Contract

Run LakeLogic using the YAML contract. Update credentials as needed.


In [ ]:
from pathlib import Path
from lakelogic import DataProcessor

BASE = Path.cwd()
contract_path = BASE / "parquet_contract.yaml"
if not contract_path.exists():
    candidate = BASE / "examples" / "03_data_sources" / "files" / "parquet" / "parquet_contract.yaml"
    if candidate.exists():
        BASE = candidate.parent
        contract_path = candidate

data_dir = BASE / "data"
data_dir.mkdir(parents=True, exist_ok=True)

parquet_path = data_dir / "sample.parquet"
if not parquet_path.exists():
    seed_contract = {
        "version": "1.0.0",
        "dataset": "seed_parquet",
        "materialization": {
            "strategy": "overwrite",
            "target_path": str(parquet_path),
            "format": "parquet",
        },
    }
    seed_rows = [
        {"id": 1, "email": "alice@example.com"},
        {"id": 2, "email": ""},
        {"id": 3, "email": "bob@example.com"},
    ]
    try:
        DataProcessor(contract=seed_contract).run(seed_rows, materialize=True)
    except Exception as exc:
        print(f"Unable to create sample parquet file: {exc}")

if not parquet_path.exists():
    print(f"Parquet file not available: {parquet_path.resolve()}")
else:
    processor = DataProcessor(contract=contract_path)
    result = processor.run_source(parquet_path)

    print(result)
    print(f"Good: {len(result.good)} | Bad: {len(result.bad)}")
